In [1]:
import math
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
import torchmetrics
import pytorch_lightning as pl

In [2]:
from od_datasets import TomatoDataset


In [3]:
# hyperparams
in_channels = 3
num_classes = 4
learning_rate = 0.001
batch_size = 4
num_epochs = 5

In [4]:
image_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

In [6]:
# train_dataset = datasets.ImageFolder("/Users/pepe/dev/upb/topics/datasets/cats/train", transform=image_transforms)
train_dataset = TomatoDataset(
    root="C:\\Users\\enriq\\Documents\\MyDocuments\\University\\S8\\TopicosAI\\dataset\\tomatos\\images\\train", 
    annotation="C:\\Users\\enriq\\Documents\\MyDocuments\\University\\S8\\TopicosAI\\dataset\\tomatos\\annotations\\tomatOD_train.json",
    transforms=image_transforms
    )
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, persistent_workers=True)
# test_dataset = datasets.ImageFolder("/Users/pepe/dev/upb/topics/datasets/cats/test", transform=image_transforms)
test_dataset = TomatoDataset(
    root="C:\\Users\\enriq\\Documents\\MyDocuments\\University\\S8\\TopicosAI\\dataset\\tomatos\\images\\val", 
    annotation="C:\\Users\\enriq\\Documents\\MyDocuments\\University\\S8\\TopicosAI\\dataset\\tomatos\\annotations\\tomatOD_test.json",
    transforms=image_transforms
    )
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, num_workers=24, persistent_workers=True)


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


c:\Users\enriq\Documents\MyDocuments\University\S8\TopicosAI\repos\my-ai-topics-2-2024\.venv\Lib\site-packages\torch\utils\data\dataloader.py:557: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
from typing import Any


from pytorch_lightning.utilities.types import STEP_OUTPUT


class MyRetinaNet(pl.LightningModule):
    def __init__(self, num_classes, freeze_backbone=False):
        super().__init__()
        self.weights = torchvision.models.detection.RetinaNet_ResNet50_FPN_V2_Weights.DEFAULT
        self.model = torchvision.models.detection.retinanet_resnet50_fpn_v2(weights=self.weights)
        in_features = self.model.backbone.out_channels
        num_anchors = self.model.head.classification_head.num_anchors
        self.model.head.classification_head.num_classes = num_classes
        cls_logits = torch.nn.Conv2d(in_features, num_anchors * num_classes, kernel_size=3, stride=1, padding=1)
        torch.nn.init.normal_(cls_logits.weight, std=0.01)
        torch.nn.init.constant_(cls_logits.bias, -math.log((1 - 0.01) / 0.01))
        self.model.head.classification_head.cls_logits = cls_logits
        # self.preprocess = self.weights.transforms()
        

    def forward(self, x, target=None):
        return self.model(x, target)
    
    def training_step(self, batch, batch_idx):
        images, targets = batch
        targets = [{k: v for k,v in t.items()} for t in targets]
        images = torch.stack(images).float()
        
        loss_dict = self.model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        return {"loss": losses, "log": loss_dict, "progress_bar": loss_dict}
        
    def validation_step(self, batch, batch_idx):
        images, targets = batch
        targets = [{k: v for k, v in t.items()} for t in targets]
        # separate losses
        images = torch.stack(images).float()
        loss_dict = self.model(images, targets)
        # Calculate Total Loss
        loss = sum(loss for loss in loss_dict.values())
        loss = torch.as_tensor(loss)
        logs = {"val_loss": loss}
        return {"val_loss": loss, "log": logs,"progress_bar": logs,}
    
    def test_step(self, batch, batch_idx, *args, **kwargs):
        images, targets, _ = batch
        targets = [{k: v for k, v in t.items()} for t in targets]
        outputs = self.net.predict(images)
        res = {t["image_id"].item(): o for t, o in zip(targets, outputs)}
        self.test_evaluator.update(res)
        return {}

    def test_epoch_end(self, outputs, *args, **kwargs):
        self.test_evaluator.accumulate()
        self.test_evaluator.summarize()
        metric = self.test_evaluator.coco_eval["bbox"].stats[0]
        metric = torch.as_tensor(metric)
        logs = {"AP": metric}
        return {"AP": metric,"log": logs, "progress_bar": logs,}

    def configure_optimizers(self):
        return torch.optim.AdamW(self.model.parameters(), lr=1e-4)

In [8]:
model = MyRetinaNet(num_classes=num_classes)
x = torch.randn(1, 3, 224, 224)
model.eval()
print(model(x))

[{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>), 'scores': tensor([], grad_fn=<IndexBackward0>), 'labels': tensor([], dtype=torch.int64)}]


In [9]:
torch.set_float32_matmul_precision('high')
trainer = pl.Trainer(
    accelerator="cpu", # para GPUs Nvidia: "gpu"
    devices=1,          
    min_epochs=1,
    max_epochs=num_epochs,
    precision="bf16-mixed"
)
trainer.fit(model, train_loader, test_loader)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\enriq\Documents\MyDocuments\University\S8\TopicosAI\repos\my-ai-topics-2-2024\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name  | Type      | Params | Mode
-------------------------------------------
0 | model | RetinaNet | 36.4 M | eval
-------------------------------------------
36.2 M    Trainable params
225 K     Non-trainable params
36.4 M    Total params
1

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\enriq\Documents\MyDocuments\University\S8\TopicosAI\repos\my-ai-topics-2-2024\.venv\Lib\site-packages\torch\utils\data\_utils\worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\enriq\Documents\MyDocuments\University\S8\TopicosAI\repos\my-ai-topics-2-2024\.venv\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\enriq\Documents\MyDocuments\University\S8\TopicosAI\repos\my-ai-topics-2-2024\.venv\Lib\site-packages\torch\utils\data\_utils\collate.py", line 317, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\enriq\Documents\MyDocuments\University\